In [19]:
import praw

In [20]:
reddit = praw.Reddit(client_id='4hzFNLs3DXmWcA',
                     client_secret='0JO0s7Nn8bolJYme3wjJmB_Zwm8',
                     user_agent='script:4hzFNLs3DXmWcA:1.0.0 (by /u/SomeGuyNamedGuy)')

In [21]:
print(reddit.read_only)

True


In [22]:
subreddit_ph = reddit.subreddit('Philippines+SafeSexPH')
subreddit_ph

Subreddit(display_name='Philippines+SafeSexPH')

In [66]:
post_list = subreddit_ph.search(query="title:condom OR selftext:condom", limit=10)
post_list

In [67]:
index = 0
for submission in post_list:
    if index == 0:
        print(submission.comments[0].body)
        index += 1

They are LITERALLY next to the kahera.


In [24]:
submission = reddit.submission(url='https://www.reddit.com/r/Philippines/comments/c98x49/grabe_pare_kaunti_na_lang_talaga_mga_maginoo_na/')
submission.comments.replace_more(limit=None)
submission

Submission(id='c98x49')

### Class-Based Tree Initialization

In [7]:
class CommentClass:
    parent = None
    children = list()
    obj = None
    depth = 1
    
    def __init__(self, obj, parent):
        self.obj = obj
        if parent:
            self.parent = parent
            self.parent.add_child(self)
            self.depth = self.parent.depth + 1
    
    def add_child(self, child):
        self.children.append(child)

In [30]:
# Recursively goes through the comment thread starting with the parent node
def make_tree(curr_comment, parent):
    curr_class = CommentClass(curr_comment, parent)
    for child in curr_comment.replies:
        print(child.body)
        make_tree(child, curr_class)
    
    return curr_class

In [31]:
top_comment_classes = list()
for comment in submission.comments[:1]:
    comment_class = make_tree(comment, None)
    top_comment_classes.append(comment_class)

lol this is my favorite line from those insecure males. Like, "ang tunay na lalake manual ang dina-drive", "ang tunay na lalake walang abs", etc. 

They're so insecure of their sexuality that they correlate it to something that can be done by anyone and think they're so manly.
Ang tunay na lalake laging may extra rice!
John Lloyd. Tunay na lalake.
Ang tunay na lalake laging may dalang payong
Ang tunay na lalaki ay bakla...

Wait no I think that's from a gloc 9 song, nvm
linyahang white knights
r/gatekeeping din hahahah


In [37]:
top_comment_classes[0].children[2].obj.body

'John Lloyd. Tunay na lalake.'

#### Disregard above, as PRAW already has built-in tree structure in comments

In [38]:
comment1 = submission.comments[0]

In [47]:
comment1.replies[0].replies[0].replies[0].depth

3

## Doc2Vec

In [91]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec

In [92]:
df = pd.read_csv('labelled_dataset.csv')
df.head()

,Unnamed: 0,labels,body,parent,submission
0,0,supporting,no question want thank post topic birth contro...,updated september scroll downhi lengthy post c...,updated september scroll downhi lengthy post c...
1,1,supporting,holy shit thank much someone recently turn sup...,updated september scroll downhi lengthy post c...,updated september scroll downhi lengthy post c...
2,2,commenting,people nice friendly super non judgmental espe...,updated september scroll downhi lengthy post c...,updated september scroll downhi lengthy post c...
3,3,supporting,thanks post tried yaz oral contraceptive prior...,updated september scroll downhi lengthy post c...,updated september scroll downhi lengthy post c...
4,4,questioning,minor need show parent consent,updated september scroll downhi lengthy post c...,updated september scroll downhi lengthy post c...


In [93]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
comments = [TaggedDocument(row[1][2], [row[1][0]]) for row in df.iterrows()]
source = [TaggedDocument(row[1][3], [row[1][0]]) for row in df.iterrows()]
model_comments = Doc2Vec(comments, vector_size=5, window=2, min_count=1, workers=4)
model_source = Doc2Vec(source, vector_size=5, window=2, min_count=1, workers=4)

In [94]:
#model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

In [98]:
import numpy as np

labels = df['labels']
final_vec_list = [np.append(model_comments.docvecs[i], model_source.docvecs[i]) for i in range(len(model_comments.docvecs))]

In [90]:
from sklearn.model_selection import train_test_split

text_train, text_test, label_train, label_test = train_test_split(final_vec_list, labels, test_size=0.2)

In [26]:
from gensim.test.utils import get_tmpfile

fname = get_tmpfile("my_doc2vec_model")

model.save(fname)
model = Doc2Vec.load(fname)  # you can continue training with the loaded model!

In [27]:
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)